In [1]:
from os import mkdir
from os.path import exists, join
from shutil import rmtree, copytree
from yaml import dump

In [2]:
DATASET = "dataset"
YAML = "data.yaml"
ORIGEM = join("Boundingbox", "test")
DESTINO = DATASET

In [3]:
if not exists(DATASET):
    mkdir(DATASET)
else:
    rmtree(DATASET)
    mkdir(DATASET)

In [4]:
def creat_yaml(names: dict = {"0": "FAMACHA"}):
    yaml_configs = {
        "train": "train/",
        "val": "valid/",
        "test": "test/",
        "names": names
    }
    return yaml_configs

yaml_configs = creat_yaml()


In [5]:
with open(join(DATASET, YAML), "w") as file:
    dump(yaml_configs, file)

In [6]:
def data(roboflow_dataset, output_dataset):
    if exists(roboflow_dataset):
        for folder in ["test", "train", "valid"]:
            item = join(roboflow_dataset, folder)
            if exists(item):
                copytree(item, join(output_dataset,folder))

data("Boundingbox", DATASET)

In [7]:
#remove(DATASET)
#removedirs(DATASET)
#rmdir(DATASET)
